## OpenMeteo-API

In [3]:
!pip install openmeteo-requests
!pip install requests-cache retry-requests numpy pandas

In [ ]:
import pandas as pd
import requests
import time
from datetime import datetime

# CSV 파일 로드
df = pd.read_csv("../vessel_data/벌커선_IMO_2197316_통합데이터.csv")

# 날짜 컬럼만 추출하여 중복 제거
date_list = pd.to_datetime(df['일자']).dt.date.drop_duplicates().sort_values().tolist()

# 고정 위경도 설정
fixed_lat = -21.19508
fixed_lon = 149.60326

# API 설정
url = "https://archive-api.open-meteo.com/v1/archive"
hourly_vars = [
    "temperature_2m", "relative_humidity_2m", "dew_point_2m", "apparent_temperature",
    "precipitation", "rain", "snowfall", "snow_depth", "weather_code", "pressure_msl",
    "surface_pressure", "cloud_cover", "cloud_cover_low", "cloud_cover_mid", "cloud_cover_high",
    "et0_fao_evapotranspiration", "vapour_pressure_deficit", "wind_speed_10m", "wind_speed_100m",
    "wind_direction_10m", "wind_direction_100m", "wind_gusts_10m"
]

results = []

for idx, date in enumerate(date_list):
    date_iso = date.isoformat()
    params = {
        "latitude": fixed_lat,
        "longitude": fixed_lon,
        "start_date": date_iso,
        "end_date": date_iso,
        "hourly": ",".join(hourly_vars),
        "timezone": "auto"
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()

        # 시간 파싱
        times = data.get("hourly", {}).get("time", [])
        if not times:
            print(f"❌ No data for date {date}")
            continue

        hourly_data = {"datetime": pd.to_datetime(times)}
        for var in hourly_vars:
            hourly_data[var] = data["hourly"].get(var, [None] * len(times))

        df_weather = pd.DataFrame(hourly_data)
        df_weather["위도"] = fixed_lat
        df_weather["경도"] = fixed_lon
        df_weather["요청일자"] = date_iso
        results.append(df_weather)

    except Exception as e:
        print(f"❌ Error at index {idx} for {date}: {e}")

    time.sleep(1)  # 과도한 요청 방지

# 결과 병합
final_df = pd.concat(results, ignore_index=True)

# 컬럼명 매핑
column_map = {
    "datetime": "날짜시간",
    "temperature_2m": "2m 기온",
    "relative_humidity_2m": "2m 상대습도",
    "dew_point_2m": "이슬점 온도",
    "apparent_temperature": "체감 온도",
    "precipitation": "강수량",
    "rain": "비",
    "snowfall": "적설량",
    "snow_depth": "적설 깊이",
    "weather_code": "날씨 코드",
    "pressure_msl": "해수면 기압",
    "surface_pressure": "지면 기압",
    "cloud_cover": "전운량",
    "cloud_cover_low": "하층 운량",
    "cloud_cover_mid": "중층 운량",
    "cloud_cover_high": "상층 운량",
    "et0_fao_evapotranspiration": "증발산량",
    "vapour_pressure_deficit": "수증기압 부족량",
    "wind_speed_10m": "10m 풍속",
    "wind_speed_100m": "100m 풍속",
    "wind_direction_10m": "10m 풍향",
    "wind_direction_100m": "100m 풍향",
    "wind_gusts_10m": "10m 돌풍속도",
    "위도": "위도",
    "경도": "경도",
    "요청일자": "요청일자"
}

final_df.rename(columns=column_map, inplace=True)

# 결과 미리보기
print(final_df.head())
final_df.to_csv("../vessel_data/meteo_weather_data.csv")

             datetime  temperature_2m  relative_humidity_2m  dew_point_2m  \
0 2023-03-22 00:00:00            24.7                    88          22.5   
1 2023-03-22 01:00:00            24.5                    89          22.6   
2 2023-03-22 02:00:00            24.4                    90          22.6   
3 2023-03-22 03:00:00            24.5                    89          22.5   
4 2023-03-22 04:00:00            24.5                    89          22.5   

   apparent_temperature  precipitation  rain  snowfall snow_depth  \
0                  28.4            0.0   0.0       0.0       None   
1                  27.9            0.0   0.0       0.0       None   
2                  27.7            0.0   0.0       0.0       None   
3                  27.7            0.0   0.0       0.0       None   
4                  27.7            0.0   0.0       0.0       None   

   weather_code  ...  et0_fao_evapotranspiration  vapour_pressure_deficit  \
0             2  ...                        0

,날짜시간,2m 기온,2m 상대습도,이슬점 온도,체감 온도,강수량,비,적설량,적설 깊이,날씨 코드,...,증발산량,수증기압 부족량,10m 풍속,100m 풍속,10m 풍향,100m 풍향,10m 돌풍속도,위도,경도,요청일자
0,2023-03-22 00:00:00,24.7,88,22.5,28.4,0.0,0.0,0.0,None,2,...,0.01,0.38,10.6,10.8,242,240,17.3,-25.728001,-48.309898,2023-03-22
1,2023-03-22 01:00:00,24.5,89,22.6,27.9,0.0,0.0,0.0,None,3,...,0.01,0.32,12.7,12.9,241,240,17.3,-25.728001,-48.309898,2023-03-22
2,2023-03-22 02:00:00,24.4,90,22.6,27.7,0.0,0.0,0.0,None,3,...,0.01,0.31,13.7,13.9,235,233,20.9,-25.728001,-48.309898,2023-03-22
3,2023-03-22 03:00:00,24.5,89,22.5,27.7,0.0,0.0,0.0,None,3,...,0.01,0.34,13.1,13.3,233,232,19.8,-25.728001,-48.309898,2023-03-22
4,2023-03-22 04:00:00,24.5,89,22.5,27.7,0.0,0.0,0.0,None,3,...,0.01,0.35,14.0,14.5,222,222,19.8,-25.728001,-48.309898,2023-03-22
